In [3]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

1255
28361


In [4]:
#首先统计所有比赛中出现的选手，确定选手顺序，准备选手矩阵表
summoner_set = Set()
for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx % 1000 == 0:
        print a_idx
    participantIdentities = a_match['participantIdentities']
    for a_identity in participantIdentities:
        a_summonerId = a_identity['player']['summonerId']
        summoner_set.add(a_summonerId)
print len(summoner_set)
ordered_summoner_set = sorted(summoner_set)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
25116


In [8]:
mat_size = len(ordered_summoner_set)
summoner_id_2_idx = {}
summoner_idx_2_id = {}
for a_idx, a_id in enumerate(ordered_summoner_set):
    if a_idx % 10000 == 0:
        print a_idx
    summoner_id_2_idx[a_id] = a_idx
    summoner_idx_2_id[a_idx] = a_id

0
10000
20000


In [10]:
def FillMat(mat, summonerId, team):
    for a_member in team:
        if summonerId != a_member:
            mat[a_member][summonerId] += 1
            mat[summonerId][a_member] += 1

In [9]:
def GetWinningSide(a_match):
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    return winning_side

In [11]:
against_mat_num = np.zeros((mat_size, mat_size))
against_mat_win = np.zeros((mat_size, mat_size))
synergy_mat_num = np.zeros((mat_size, mat_size))
synergy_mat_win = np.zeros((mat_size, mat_size))

for a_idx, a_match in enumerate(matches_collection.find()):
    if a_idx % 1000 == 0:
        print a_idx
    participantIdentities = a_match['participantIdentities']
    blue_team = []
    red_team = []
    for a_identity in participantIdentities:
        a_summonerId = a_identity['player']['summonerId']
        a_participantId = a_identity['participantId']
        if a_participantId <= 5:
            blue_team.append(a_summonerId)
        else:
            red_team.append(a_summonerId)
            
    for a_identity in participantIdentities:
        a_summonerId = a_identity['player']['summonerId']
        a_participantId = a_identity['participantId']
        if a_participantId <= 5:
            FillMat(against_mat_num, a_summonerId, red_team)
            FillMat(synergy_mat_num, a_summonerId, blue_team)
        else:
            FillMat(against_mat_num, a_summonerId, blue_team)
            FillMat(synergy_mat_num, a_summonerId, red_team)
            
    winning_side == GetWinningSide(a_match)
    if winning_side == 0:
        for a_blue in blue_team:
            FillMat(against_mat_win, a_blue, red_team)
            FillMat(synergy_mat_win, a_blue, blue_team)
    if winning_side == 1:
        for a_red in red_team:
            FillMat(against_mat_win, a_red, blue_team)
            FillMat(synergy_mat_win, a_red, red_team)

MemoryError: 